<a href="https://colab.research.google.com/github/kenpurin/Retrosynthesis_Pytorch/blob/main/score_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2021-06-17 23:07:25--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   212MB/s    in 0.4s    

2021-06-17 23:07:26 (212 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b64

In [4]:
!git clone https://github.com/kenpurin/Retrosynthesis_TF.git

fatal: destination path 'Retrosynthesis_TF' already exists and is not an empty directory.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip -q install rdkit-pypi

     |████████████████████████████████| 18.6MB 169kB/s 


In [3]:
from rdkit import Chem
import pandas as pd

In [ ]:
invalid_smiles_t= True
beam_size=10
#opt_predictions='/content/predictions_USPTO-50K_model_full_version_step_500000_beam10.txt'
opt_targets='/content/gdrive/MyDrive/code retrosynthesis/score_test/tgt-test.txt'
opt_predictions="/content/gdrive/MyDrive/code retrosynthesis/score_test/predictions_top10_test2000_beam_new.txt"

In [ ]:
def canonicalize_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        return Chem.MolToSmiles(mol, isomericSmiles=True)
    else:
        return ''

In [ ]:
def get_rank(row, base, max_rank):
  for i in range(1, max_rank+1):
    #print(row['{}{}'.format(base, i)])
    if row['target'] == row['{}{}'.format(base, i)]:
      return i
  return 0

In [ ]:
with open(opt_targets, 'r') as f:
    targets = [''.join(line.strip().split(' ')) for line in f.readlines()]
targets = targets[:]
predictions = [[] for i in range(beam_size)]
test_df = pd.DataFrame(targets)
test_df.columns = ['target']
total = len(test_df)
total

5004

In [ ]:
with open(opt_predictions, 'r') as f:
  for i, line in enumerate(f.readlines()):
    predictions[i % beam_size].append(''.join(line.strip().split(' ')))

In [ ]:
for i, preds in enumerate(predictions):
    test_df['prediction_{}'.format(i + 1)] = preds
    test_df['canonical_prediction_{}'.format(i + 1)] = test_df['prediction_{}'.format(i + 1)].apply(
    lambda x: canonicalize_smiles(x))  

In [ ]:
test_df

,target,prediction_1,canonical_prediction_1,prediction_2,canonical_prediction_2,prediction_3,canonical_prediction_3,prediction_4,canonical_prediction_4,prediction_5,canonical_prediction_5,prediction_6,canonical_prediction_6,prediction_7,canonical_prediction_7,prediction_8,canonical_prediction_8,prediction_9,canonical_prediction_9,prediction_10,canonical_prediction_10
0,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)c(O)c1,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)c(O)c1,C1=COCCC1.COC(=O)CC(=O)c1ccc(O)cc1O,C1=COCCC1.COC(=O)CC(=O)c1ccc(O)cc1O,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)sc1O,,C1=COCCC1.COC(=O)CCC(=O)c1csc(O)cc1O,,C1=COCCC1.CO(=O)CCC(=O)c1ccc(O)cc1O,,COC(=O)CCC(=O)c1ccc(O)cc1O.C1=COCCC1,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,C1=COCCC1.COC(=O)CCC(=O)c1cc(O)ccc1O,C1=COCCC1.COC(=O)CCC(=O)c1cc(O)ccc1O,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1OC,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1OC,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)c(O)c1O,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)c(O)c1O
1,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ncccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cnccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ccncc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ccccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ccccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ccncc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ccncc1N,COC(=O)c1ccCc(C(=O)O)c1.Nc1ncccc1N,,COC(=O)c1cccc(C#N)c1.Nc1ncccc1N,COC(=O)c1cccc(C#N)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ncccn1N,,COC(=O)c1cccC(C(=O)O)c1.Nc1ncccc1N,,COC(=O)c1ccCc(C(=O)O)c1.Nc1cnccc1N,
2,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1HCC(C(=O)O)CC1.CNOC,,CC(C)(C)OC(=O)NC1CCC(C(=O)O)(CC1)CC1.CNOC,,CC(C)(C)OC(=O)NC1CCC(C(=O)O)Cc1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CC=C(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CC=C(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)(C)C)CC1.CNOC,,CC(C)(C)OC(=O)NC1CCC(C(=O)O)(C(=O)O)CC1,CC(C)(C)OC(=O)NC1CCC(C(=O)O)(C(=O)O)CC1,CC(C)(C)OC(=O)NC1CCC(C(=O)O)(CC1)CC1.CONC,,CC(C)(C)OC(=O)NC1CCC(C(=O)O)(C)C)CC1.CONC,
3,Nc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1cc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,,Nc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1l,,Nc1ccC(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,,Nc1cc(O)ccc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1cccc(O)c1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1ccn(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,,Nc1cc[nH]cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,,Nc1cccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,,Nc1ncc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1ccc(O)cn1.O=[N+]([O-])c1ccc(Cl)nc1Cl,,
4,[N-]=[N+]=NCC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([...,N.O=[N+]([O-])[C@H]1CC=C(CBr)CC[C@H]1c1ccc(Cl)...,N.O=[N+]([O-])[C@H]1CC=C(CBr)CC[C@H]1c1ccc(Cl)...,N.O=[N+]([O-])[C@H]1CC=C(CBr)=CC[C@H]1c1ccc(Cl...,,N.O=[N+]([O-])[C@H]1CC=C(CBr)C[C@H]1c1ccc(Cl)c...,N.O=[N+]([O-])[C@H]1CC=C(CBr)C[C@H]1c1ccc(Cl)c...,N.O=[N+]([O-])[C@@H]1CC=C(CBr)CC[C@H]1c1ccc(Cl...,N.O=[N+]([O-])[C@@H]1CC=C(CBr)CC[C@H]1c1ccc(Cl...,N.O=[N+]([O-])[C@H]1CC=C(CBr)Cc2ccc(Cl)cc2Cl)C1,,N.O=[N+]([O-])C1CC=C(CBr)C[C@H]1c1ccc(Cl)cc1Cl,N.O=[N+]([O-])C1CC=C(CBr)C[C@H]1c1ccc(Cl)cc1Cl,N.O=[N+]([O-])C1CC=C(CBr)CC[C@H]1c1ccc(Cl)cc1Cl,N.O=[N+]([O-])C1CC=C(CBr)CC[C@H]1c1ccc(Cl)cc1Cl,N.O=[N+]([O-])[C@H]1CC=C(CCl)CC[C@H]1c1ccc(Cl)...,N.O=[N+]([O-])[C@H]1CC=C(CCl)CC[C@H]1c1ccc(Cl)...,[N-]=[N+]=NCC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([...,[N-][C@@H]1CC(CN=[N+]=[N-])=CC[C@H]1c1ccc(Cl)c...,N.O=[N+]([O-])[C@H]1CC=C(CBr)Cc2ccc(Cl)cc2Cl)C...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999,Cc1cc([N+](=O)[O-])ccc1O.Nc1cc(Cl)ccn1,Cc1cc([N+](=O)[O-])ccc1F.Nc1cc(O)ccn1,Cc1cc([N+](=O)[O-])ccc1F.Nc1cc(O)ccn1,Cc1cc([N+](=O)[O-])ccc1O.Nc1cc(Cl)ccn1,Cc1cc([N+](=O)[O-])ccc1O.Nc1cc(Cl)ccn1,Cc1cc([N+](=O)[O-]

In [ ]:
test_df['rank'] = test_df.apply(lambda row: get_rank(row, 'canonical_prediction_', beam_size), axis=1)
test_df['rank']

0       1
1       1
2       1
3       1
4       0
       ..
4999    2
5000    0
5001    1
5002    0
5003    5
Name: rank, Length: 5004, dtype: int64

In [ ]:
correct = 0
invalid_smiles = 0

for i in range(1, beam_size+1):
    correct += (test_df['rank'] == i).sum()
    print((test_df['rank'] == i).sum(),correct)
    
    invalid_smiles += (test_df['canonical_prediction_{}'.format(i)] == '').sum()
    if invalid_smiles_t:
        print('Top-{}: {:.1f}% || Invalid SMILES {:.2f}%'.format(i, correct/total*100,
                                                                 invalid_smiles/(total*i)*100))
    else:
        print('Top-{}: {:.1f}%'.format(i, correct / total * 100))

2747 2747
Top-1: 54.9% || Invalid SMILES 3.04%
503 3250
Top-2: 64.9% || Invalid SMILES 9.34%
184 3434
Top-3: 68.6% || Invalid SMILES 14.16%
109 3543
Top-4: 70.8% || Invalid SMILES 17.78%
65 3608
Top-5: 72.1% || Invalid SMILES 20.49%
46 3654
Top-6: 73.0% || Invalid SMILES 22.79%
42 3696
Top-7: 73.9% || Invalid SMILES 24.90%
30 3726
Top-8: 74.5% || Invalid SMILES 26.76%
18 3744
Top-9: 74.8% || Invalid SMILES 28.54%
10 3754
Top-10: 75.0% || Invalid SMILES 30.84%


In [ ]:
2747 2747
Top-1: 54.9% || Invalid SMILES 3.04%
503 3250
Top-2: 64.9% || Invalid SMILES 9.34%
184 3434
Top-3: 68.6% || Invalid SMILES 14.16%
109 3543
Top-4: 70.8% || Invalid SMILES 17.78%
65 3608
Top-5: 72.1% || Invalid SMILES 20.49%
46 3654
Top-6: 73.0% || Invalid SMILES 22.79%
42 3696
Top-7: 73.9% || Invalid SMILES 24.90%
30 3726
Top-8: 74.5% || Invalid SMILES 26.76%
18 3744
Top-9: 74.8% || Invalid SMILES 28.54%
10 3754
Top-10: 75.0% || Invalid SMILES 30.84%

SyntaxError: ignored

In [ ]:
2783 2783
Top-1: 55.6% || Invalid SMILES 3.52%
224 3007
Top-2: 60.1% || Invalid SMILES 17.67%
110 3117
Top-3: 62.3% || Invalid SMILES 22.40%
61 3178
Top-4: 63.5% || Invalid SMILES 25.21%
50 3228
Top-5: 64.5% || Invalid SMILES 26.94%
31 3259
Top-6: 65.1% || Invalid SMILES 28.10%
37 3296
Top-7: 65.9% || Invalid SMILES 29.11%
29 3325
Top-8: 66.4% || Invalid SMILES 29.79%
20 3345
Top-9: 66.8% || Invalid SMILES 30.32%
16 3361
Top-10: 67.2% || Invalid SMILES 30.52%